In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [2]:
seed = 42
tf.random.set_seed(seed)

In [3]:
data = pd.read_csv('..\\Data\\ze41_mol_desc_db_red.csv', header=0, sep=';', decimal=',')

In [4]:
col_names = data.columns
#x_cols = ['P_VSA_MR_5', 'Mor04m', 'LUMO / eV']
x_cols = ['P_VSA_MR_5', 'Mor04m', 'E1p', 'Mor22s', 'LUMO / eV']
#x_cols=['VE2_G/D', 'Eig14_EA(dm)', 'Mor31m', 'TDB04u', 'HATS1e']
#x_cols=['VE2_G/D', 'Eig14_EA(dm)', 'Mor31m']
#x_cols = ['CATS2D_04_DP', 'CATS3D_02_AP', 'Eta_betaS', 'P_VSA_LogP_4', 'VE1_B(s)']
#x_cols = ['B04[C-C]', 'CATS3D_03_DP', 'Eig11_AEA(ed)', 'J_RG', 'SpMAD_EA(ed)']
X = data[x_cols]
y = data[col_names[2]]

In [5]:
X_scaled = MinMaxScaler(feature_range=(0,1)).fit_transform(X)

In [6]:
X_train, X_valid, y_train, y_valid = train_test_split(X_scaled, y, test_size=0.1, random_state=seed)

In [7]:
model = keras.models.Sequential([
    keras.layers.Dense(50, activation='relu', input_shape=(len(x_cols),)),
    keras.layers.Dense(20, activation='relu'),
    keras.layers.Dense(10, activation='relu'),
    keras.layers.Dense(1)
])

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 50)                300       
_________________________________________________________________
dense_1 (Dense)              (None, 20)                1020      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                210       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 11        
Total params: 1,541
Trainable params: 1,541
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.01),
    loss='mean_squared_error')

In [10]:
history = model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=25)

Epoch 1/25
2/2 [==============================] - 0s 59ms/step - loss: 0.4029 - val_loss: 0.2034
Epoch 2/25
2/2 [==============================] - 0s 4ms/step - loss: 0.1603 - val_loss: 0.0512
Epoch 3/25
2/2 [==============================] - 0s 4ms/step - loss: 0.1270 - val_loss: 0.0511
Epoch 4/25
2/2 [==============================] - 0s 4ms/step - loss: 0.1376 - val_loss: 0.0351
Epoch 5/25
2/2 [==============================] - 0s 4ms/step - loss: 0.0884 - val_loss: 0.0433
Epoch 6/25
2/2 [==============================] - 0s 4ms/step - loss: 0.0814 - val_loss: 0.0673
Epoch 7/25
2/2 [==============================] - 0s 4ms/step - loss: 0.0904 - val_loss: 0.0669
Epoch 8/25
2/2 [==============================] - 0s 4ms/step - loss: 0.0883 - val_loss: 0.0460
Epoch 9/25
2/2 [==============================] - 0s 4ms/step - loss: 0.0731 - val_loss: 0.0253
Epoch 10/25
2/2 [==============================] - 0s 4ms/step - loss: 0.0638 - val_loss: 0.0175
Epoch 11/25
2/2 [=====================

In [11]:
y_pred = model.predict(X_valid)

In [12]:
for i in range(y_pred.shape[0]):
    print('true: {}, predicted: {:.3f}'.format(y_valid.iloc[i], y_pred[i, 0]))

true: 0.328, predicted: 0.471
true: 0.896, predicted: 0.808
true: 0.817, predicted: 0.940
true: 0.765, predicted: 0.728
true: 0.893, predicted: 1.137
true: 0.733, predicted: 0.746


In [13]:
y_valid

0     0.328
5     0.896
36    0.817
45    0.765
13    0.893
54    0.733
Name: LinIE ZE41, dtype: float64

In [14]:
y_pred*345-270

array([[-107.337845],
       [   8.827606],
       [  54.15686 ],
       [ -18.731384],
       [ 122.156494],
       [ -12.51355 ]], dtype=float32)